In [1]:
import sys
sys.path.insert(0, '../../')  

import math
import numpy as np
import matplotlib.pyplot as plt 
from gasp import train_gasp, run_gasp, simulate_ssfp, simulate_ssfp_simple, sampling, responses, SSFPParams

In [2]:
def run_model(D, params):
    width = 256
    height = 1
    gradient = 2 * np.pi
    method = 'affine'
    T2base = 0.01
    T1T2ratio = 5

    M = simulate_ssfp_simple(width=width, height=height, T1=T1T2ratio * T2base, T2 = T2base, params=params, minTR = np.min(params.TRs), gradient = gradient)
    _, A = train_gasp(M, D, method=method)
    I = run_gasp(M, A, method=method)
    MSE = np.sqrt(np.mean((np.abs(I) - D)**2))

    return A, MSE

width = 256
npcs = 16
TRs = [5e-3, 10e-3, 20e-3]
n_points = npcs * len(TRs)
alpha = np.deg2rad(20)
TRs, PCs = sampling.grid_TR_sampling(n_points=n_points, TRs=TRs)
params = SSFPParams(n_points, alpha, TRs, PCs)

D = responses.gaussian(width, bw=0.2, shift=0)
A, MSE = run_model(D, params)
print(MSE)


0.007546711504891278


In [12]:
import numpy as np
from scipy import optimize
from typing import List, Tuple
from gasp import train_gasp, run_gasp, simulate_ssfp_simple, sampling, responses, SSFPParams

def run_model(D: np.ndarray, params: SSFPParams) -> Tuple[np.ndarray, float]:
    width, height = 256, 1
    gradient = 2 * np.pi
    method = 'affine'
    T2base = 0.01
    T1T2ratio = 5

    M = simulate_ssfp_simple(width=width, height=height, T1=T1T2ratio * T2base, T2=T2base,
                                        params=params, minTR=np.min(params.TRs), gradient=gradient)
    _, A = train_gasp(M, D, method=method)
    I = run_gasp(M, A, method=method)
    MSE = np.sqrt(np.mean((np.abs(I) - D)**2))

    return A, MSE

def objective_function(x: np.ndarray, D: np.ndarray, n_points: int) -> float:
    alphas = x[:n_points]
    TRs = x[n_points:2*n_points]
    pcs = x[2*n_points:]
    params = SSFPParams(n_points, alphas, TRs, pcs)
    _, MSE = run_model(D, params)
    return MSE

def optimize_ssfp_params(D: np.ndarray, n_points: int, 
                         initial_alphas: np.ndarray, 
                         initial_TRs: np.ndarray, 
                         initial_pcs: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    initial_guess = np.concatenate([initial_alphas, initial_TRs, initial_pcs])
    
    bounds = [(0, np.pi/2) for _ in range(n_points)]  # bounds for alphas (0 to pi/2 radians)
    bounds += [(1e-3, 100e-3) for _ in range(n_points)]  # bounds for TRs (1ms to 100ms)
    bounds += [(0, 2*np.pi) for _ in range(n_points)]  # bounds for PCs (0 to 2pi radians)

    result = optimize.minimize(objective_function, initial_guess, args=(D, n_points),
                               method='L-BFGS-B', bounds=bounds, options={'maxiter': 100, 'disp': True})

    optimal_alphas = result.x[:n_points]
    optimal_TRs = result.x[n_points:2*n_points]
    optimal_pcs = result.x[2*n_points:]
    optimal_MSE = result.fun

    return optimal_alphas, optimal_TRs, optimal_pcs, optimal_MSE

# Example usage
width = 256
n_points = 48  # total number of data points

# Initial guesses
initial_alphas = np.ones(n_points) * np.deg2rad(20)
initial_TRs, initial_pcs = sampling.grid_TR_sampling(n_points=n_points, TRs=[5e-3, 10e-3, 20e-3])

D = responses.gaussian(width, bw=0.2, shift=0)

try:
    optimal_alphas, optimal_TRs, optimal_pcs, optimal_MSE = optimize_ssfp_params(D, n_points, initial_alphas, initial_TRs, initial_pcs)

    print("Initial values:")
    print(f"Initial alphas: {np.rad2deg(initial_alphas)} degrees")
    print(f"Initial TRs: {initial_TRs * 1000} ms")
    print(f"Initial PCs: {initial_pcs} radians")

    print("\nOptimized values:")
    print(f"Optimal alphas: {np.rad2deg(optimal_alphas)} degrees")
    print(f"Optimal TRs: {optimal_TRs * 1000} ms")
    print(f"Optimal PCs: {optimal_pcs} radians")
    print(f"Optimal MSE: {optimal_MSE:.6f}")

    # Run the model with optimal parameters
    optimal_params = SSFPParams(n_points, optimal_alphas, optimal_TRs, optimal_pcs)
    A, MSE = run_model(D, optimal_params)

    print(f"Final MSE: {MSE:.6f}")

    # Calculate and print the changes
    alpha_change = np.mean(np.abs(optimal_alphas - initial_alphas))
    TR_change = np.mean(np.abs(optimal_TRs - initial_TRs))
    PC_change = np.mean(np.abs(optimal_pcs - initial_pcs))

    print("\nAverage absolute changes:")
    print(f"Alpha change: {np.rad2deg(alpha_change):.6f} degrees")
    print(f"TR change: {TR_change * 1000:.6f} ms")
    print(f"PC change: {PC_change:.6f} radians")

except Exception as e:
    print(f"An error occurred: {e}")

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          144     M =           10

At X0         3 variables are exactly at the bounds

At iterate    0    f=  7.54671D-03    |proj g|=  9.50000D-02

At iterate    1    f=  5.32698D-03    |proj g|=  9.50002D-02

At iterate    2    f=  5.07219D-03    |proj g|=  9.50002D-02

At iterate    3    f=  5.04321D-03    |proj g|=  9.50002D-02

At iterate    4    f=  5.00539D-03    |proj g|=  9.50002D-02

At iterate    5    f=  2.41117D-03    |proj g|=  9.50002D-02

At iterate    6    f=  1.07164D-03    |proj g|=  9.50005D-02

At iterate    7    f=  1.01833D-03    |proj g|=  9.50005D-02

At iterate    8    f=  8.54748D-04    |proj g|=  9.50002D-02

At iterate    9    f=  6.98361D-04    |proj g|=  9.50024D-02

At iterate   10    f=  6.10450D-04    |proj g|=  9.50041D-02

At iterate   11    f=  5.76285D-04    |proj g|=  9.50033D-02

At iterate   12    f=  5.39001D-04    |proj g|=  9.50050D-02

At iterate   13    f=  5.1